In [1]:
import sys

sys.path.append("../")
from library.datasets import ProstateCancerDataset
from library.train import create_dataloader
from entrypoint import prostate_scoring_function
import matplotlib.pyplot as plt
import torch
from library.models import *
import random
import glob
from icecream import ic
import copy
import pprint
from tqdm.auto import tqdm
import pandas as pd

In [2]:
best_prostate_models_combined = [i for i in glob.glob('../tuning_exp_training_details/pretrained_model_raw_metadata/*Combined*/**/*.pt', recursive=True)]

In [3]:
prostate_dir = "../datasets/train_prostate/train/"
random_seed = 24042

val_dataset = ProstateCancerDataset(
    prostate_dir,
    split_type="val",
    random_seed=random_seed,
    input_slice_count=3,
)
test_dataset = ProstateCancerDataset(
    prostate_dir, split_type="test", random_seed=random_seed, input_slice_count=3
)

Adding raw case data to ChaimeleonDataset:   0%|          | 0/294 [00:00<?, ?it/s]

Adding raw case data to ChaimeleonDataset:   0%|          | 0/294 [00:00<?, ?it/s]

In [4]:
val_loader = create_dataloader(val_dataset, batch_size=6, shuffle=False)
test_loader = create_dataloader(test_dataset, batch_size=6, shuffle=False)

In [5]:
model_path_to_base = {
    'ProstateCombinedResnet18PretrainedModel.pt': ProstateCombinedResnet18PretrainedModel(),
    'ProstateCombinedResnet18PretrainedModel_V2_Grid.pt': ProstateCombinedResnet18PretrainedModel_V2_Grid(),
    'ProstateCombinedResnet18PretrainedModel_V2_1_Grid.pt': ProstateCombinedResnet18PretrainedModel_V2_1_Grid(),
}
prostate_model_base = model_path_to_base['_'.join(best_prostate_models_combined[0].split('/')[-1].split('_')[3:])]

In [6]:
# load model
model_state_dict = torch.load(best_prostate_models_combined[0])
model = copy.deepcopy(prostate_model_base)
model.load_state_dict(model_state_dict)
model = model.eval()

In [7]:
# Prostate models loop
epoch_targets = []
epoch_preds = []
epoch_outputs = []
for batch_idx, (images, metadata, targets) in enumerate(val_loader):
    data = images, metadata.squeeze()
    if isinstance(targets, list):
        pfs = targets[1]
        targets = targets[0]
    else:
        targets = targets
    output = model(data)
    pred = output.argmax(dim=1, keepdim=True)
    
    targets, output, pred = (
            targets.argmax(dim=1, keepdim=True),
            torch.nn.functional.sigmoid(output),
            pred,
        )
    
    epoch_targets.extend([i.item() for i in targets])
    epoch_preds.extend([i.item() for i in pred])
    epoch_outputs.extend(
        [output[k][1].item() for k, i in enumerate(targets)]
    )

score, score_dict = prostate_scoring_function(epoch_targets, epoch_outputs, epoch_preds)

In [8]:
pprint.pprint(best_prostate_models_combined[0])
pprint.pprint(score_dict)

'../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel_V2_Grid/2023-12-18T00-34-26/fixed_augmentation_less_extreme/[]_frozen/0.0005_starting_learning_rate_0.5_factor/best_val_acc_ProstateCombinedResnet18PretrainedModel_V2_Grid.pt'
{'auc_chai': 0.5416666666666666,
 'balanced_accuracy': 0.5416666666666666,
 'roc_auc': 0.5656565656565657,
 'score': 0.5416666666666667,
 'sensitivity': 0.08333333333333333,
 'sensitivity_recall': 0.08333333333333333,
 'specificity': 1.0}


In [9]:
def load_saved_model(model_path, model_base):
    model_state_dict = torch.load(model_path)
    model = copy.deepcopy(model_base)
    model.load_state_dict(model_state_dict)
    model = model.eval()

    return model

In [10]:
def evaluate_prostate_model(eval_dataloader, model_path, model_base, verbose=False, eval_id = None):
    model = load_saved_model(model_path, model_base)
    
    epoch_targets = []
    epoch_preds = []
    epoch_outputs = []
    for batch_idx, (images, metadata, targets) in enumerate(eval_dataloader):
        data = images, metadata.squeeze()
        if isinstance(targets, list):
            pfs = targets[1]
            targets = targets[0]
        else:
            targets = targets
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)
        
        targets, output, pred = (
                targets.argmax(dim=1, keepdim=True),
                torch.nn.functional.sigmoid(output),
                pred,
            )
        
        epoch_targets.extend([i.item() for i in targets])
        epoch_preds.extend([i.item() for i in pred])
        epoch_outputs.extend(
            [output[k][1].item() for k, i in enumerate(targets)]
        )
    
    score, score_dict = prostate_scoring_function(epoch_targets, epoch_outputs, epoch_preds)
    if eval_id is not None:
        score_dict = {f"{eval_id}_{key}": val for key, val in score_dict.items()}

    model_details = model_path.split('/')[2:]

    if verbose:
        pprint.pprint(model_details)
        pprint.pprint(score_dict)
        print("\n\n")

    return score_dict

In [11]:
val_results_prostate = {}
for model_path in tqdm(best_prostate_models_combined):
    model_base_name = '_'.join(model_path.split('/')[-1].split('_')[3:])
    try:
        model_base = model_path_to_base[model_base_name]
        results = evaluate_prostate_model(val_loader, model_path, model_base, eval_id = 'val')
        val_results_prostate[model_path] = results
    except Exception as e:
        print(model_path)
        print(str(e))

  0%|          | 0/152 [00:00<?, ?it/s]

In [12]:
test_results_prostate = {}
for model_path in tqdm(best_prostate_models_combined):
    model_base_name = '_'.join(model_path.split('/')[-1].split('_')[3:])
    try:
        model_base = model_path_to_base[model_base_name]
        results = evaluate_prostate_model(test_loader, model_path, model_base, eval_id = 'test')
        test_results_prostate[model_path] = results
    except Exception as e:
        print(model_path)
        print(str(e))

  0%|          | 0/152 [00:00<?, ?it/s]

In [13]:
prostate_val_df = pd.DataFrame.from_dict(val_results_prostate, orient='index')
prostate_test_df = pd.DataFrame.from_dict(test_results_prostate, orient='index')

In [14]:
prostate_val_df

,val_score,val_roc_auc,val_auc_chai,val_balanced_accuracy,val_sensitivity,val_sensitivity_recall,val_specificity
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel_V2_Grid/2023-12-18T00-34-26/fixed_augmentation_less_extreme/[]_frozen/0.0005_starting_learning_rate_0.5_factor/best_val_acc_ProstateCombinedResnet18PretrainedModel_V2_Grid.pt,0.541667,0.565657,0.541667,0.541667,0.083333,0.083333,1.000000
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel_V2_Grid/2023-12-18T00-34-26/fixed_augmentation_less_extreme/[]_frozen/0.0005_starting_learning_rate_0.5_factor/best_val_score_ProstateCombinedResnet18PretrainedModel_V2_Grid.pt,0.704545,0.747475,0.704545,0.704545,0.833333,0.833333,0.575758
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel_V2_Grid/2023-12-18T00-34-26/fixed_augmentation_less_extreme/[]_frozen/0.01_starting_learning_rate_0.1_factor/best_val_acc_ProstateCombinedResnet18PretrainedModel_V2_Grid.pt,0.583333,0.631313,0.583333,0.583333,0.166667,0.166667,1.000000
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel_V2_Grid/2023-12-18T00-34-26/fixed_augmentation_less_extreme/[]_frozen/0.01_starting_learning_rate_0.1_factor/best_val_score_ProstateCombinedResnet18PretrainedModel_V2_Grid.pt,0.606061,0.681818,0.606061,0.606061,1.000000,1.000000,0.212121
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel_V2_Grid/2023-12-18T00-34-26/fixed_augmentation_less_extreme/[]_frozen/0.003_starting_learning_rate_0.5_factor/best_val_acc_ProstateCombinedResnet18PretrainedModel_V2_Grid.pt,0.719697,0.797980,0.719697,0.719697,0.500000,0.500000,0.939394
...,...,...,...,...,...,...,...
"../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-16T18-10-21/fixed_augmentation/['layer2', 'layer3']_frozen/0.01_starting_learning_rate_0.75_factor/best_val_acc_ProstateCombinedResnet18PretrainedModel.pt",0.715909,0.724747,0.715909,0.715909,0.583333,0.583333,0.848485
"../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-16T18-10-21/fixed_augmentation/['layer2', 'layer3']_frozen/0.01_starting_learning_rate_0.25_factor/best_val_score_ProstateCombinedResnet18PretrainedModel.pt",0.719697,0.871212,0.719697,0.719697,0.500000,0.500000,0.939394
"../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-16T18-10-21/fixed_augmentation/['layer2', 'layer3']_frozen/0.01_starting_learning_rate_0.25_factor/best_val_acc_ProstateCombinedResnet18PretrainedModel.pt",0.719697,0.871212,0.719697,0.719697,0.500000,0.500000,0.939394
"../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-16T18-10-21/fixed_augmentation/['layer2', 'layer3']_frozen/0.003_starting_learning_rate_0.75_factor/best_val_score_ProstateCombinedResnet18PretrainedModel.pt",0.746212,0.792929,0.746212,0.746212,0.916667,0.916667,0.575758


In [15]:
prostate_test_df

,test_score,test_roc_auc,test_auc_chai,test_balanced_accuracy,test_sensitivity,test_sensitivity_recall,test_specificity
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel_V2_Grid/2023-12-18T00-34-26/fixed_augmentation_less_extreme/[]_frozen/0.0005_starting_learning_rate_0.5_factor/best_val_acc_ProstateCombinedResnet18PretrainedModel_V2_Grid.pt,0.541667,0.575758,0.541667,0.541667,0.083333,0.083333,1.000000
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel_V2_Grid/2023-12-18T00-34-26/fixed_augmentation_less_extreme/[]_frozen/0.0005_starting_learning_rate_0.5_factor/best_val_score_ProstateCombinedResnet18PretrainedModel_V2_Grid.pt,0.647727,0.724747,0.647727,0.647727,0.750000,0.750000,0.545455
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel_V2_Grid/2023-12-18T00-34-26/fixed_augmentation_less_extreme/[]_frozen/0.01_starting_learning_rate_0.1_factor/best_val_acc_ProstateCombinedResnet18PretrainedModel_V2_Grid.pt,0.583333,0.623737,0.583333,0.583333,0.166667,0.166667,1.000000
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel_V2_Grid/2023-12-18T00-34-26/fixed_augmentation_less_extreme/[]_frozen/0.01_starting_learning_rate_0.1_factor/best_val_score_ProstateCombinedResnet18PretrainedModel_V2_Grid.pt,0.636364,0.656566,0.636364,0.636364,1.000000,1.000000,0.272727
../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel_V2_Grid/2023-12-18T00-34-26/fixed_augmentation_less_extreme/[]_frozen/0.003_starting_learning_rate_0.5_factor/best_val_acc_ProstateCombinedResnet18PretrainedModel_V2_Grid.pt,0.678030,0.785354,0.678030,0.678030,0.416667,0.416667,0.939394
...,...,...,...,...,...,...,...
"../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-16T18-10-21/fixed_augmentation/['layer2', 'layer3']_frozen/0.01_starting_learning_rate_0.75_factor/best_val_acc_ProstateCombinedResnet18PretrainedModel.pt",0.715909,0.729798,0.715909,0.715909,0.583333,0.583333,0.848485
"../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-16T18-10-21/fixed_augmentation/['layer2', 'layer3']_frozen/0.01_starting_learning_rate_0.25_factor/best_val_score_ProstateCombinedResnet18PretrainedModel.pt",0.693182,0.861111,0.693182,0.693182,0.416667,0.416667,0.969697
"../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-16T18-10-21/fixed_augmentation/['layer2', 'layer3']_frozen/0.01_starting_learning_rate_0.25_factor/best_val_acc_ProstateCombinedResnet18PretrainedModel.pt",0.651515,0.835859,0.651515,0.651515,0.333333,0.333333,0.969697
"../tuning_exp_training_details/pretrained_model_raw_metadata/ProstateCombinedResnet18PretrainedModel/2023-12-16T18-10-21/fixed_augmentation/['layer2', 'layer3']_frozen/0.003_starting_learning_rate_0.75_factor/best_val_score_ProstateCombinedResnet18PretrainedModel.pt",0.761364,0.757576,0.761364,0.761364,0.916667,0.916667,0.606061


In [16]:
prostate_results_df = pd.merge(prostate_val_df, prostate_test_df, left_index=True, right_index=True)

In [17]:
prostate_results_df.to_csv('./2023_12_18_prostate_eval_results.csv')

In [18]:
prostate_results_df.to_excel('./2023_12_18_prostate_eval_results_all_soft_test.xlsx', sheet_name='prostate_eval_results')